In [2]:
import pandas as pd
import requests

res=requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

with open('ratings_train.txt','wb') as f:
        f.write(res.content)

In [3]:
nsmc=pd.read_csv('ratings_train.txt',sep='\t')

In [4]:
import re

In [5]:
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [6]:
data = nsmc[nsmc['document'].notnull()]['document'].map(find_hangul)

In [7]:
data[0]

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [7]:
def only_hangul(text):
    return ' '.join(find_hangul(text))

In [8]:
data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)

In [9]:
data2[0]

'아 더빙 진짜 짜증나네요 목소리'

In [10]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(data2))

FastText 모형 학습

In [11]:
from gensim.models.fasttext import FastText
from gensim.models.word2vec import Word2Vec

In [12]:
model = FastText(size=16)

In [13]:
model.build_vocab(sentences=data)

In [14]:
model.train(
    sentences=data,
    epochs=5,
    total_examples=model.corpus_count,
    total_words=model.corpus_total_words
)

In [15]:
model.save('nsmc.fasttext')

In [16]:
model = FastText.load('nsmc.fasttext')

In [19]:
from gensim.models.fasttext import FastText
model = FastText.load('nsmc.fasttext')

단어 임베딩
'히어로'는 단어 임베딩이 학습되어 있다.

In [20]:
'히어로' in model.wv.vocab

True

In [21]:
model.wv['히어로']

array([-0.04594129,  0.45343852,  0.44429657,  0.5475346 , -0.6134086 ,
       -0.23622645, -0.78780735,  0.3909033 ,  0.22873542,  0.21316187,
       -0.71568775,  0.08368004, -0.2830746 , -0.1506468 ,  0.56481427,
       -0.7089242 ], dtype=float32)

'슈퍼히어로'는 단어 임베딩이 없지만

In [22]:
'슈퍼히어로' in model.wv.vocab

False

준단어 토큰의 임베딩을 더해서 임베딩을 계산해준다.

In [23]:
model.wv['슈퍼히어로']

array([-0.04049635,  0.26484385,  0.24594593,  0.20961554, -0.268292  ,
       -0.08478185, -0.29447132,  0.12084288,  0.11553464,  0.03385194,
       -0.3028485 ,  0.03501648, -0.10842834, -0.0880933 ,  0.23624958,
       -0.2664656 ], dtype=float32)

유사도
'히어로'와 '슈퍼히어로'의 유사도는 높다.

In [24]:
model.wv.similarity('슈퍼히어로', '히어로')

0.9847185

'히어로'와 '평론가'의 유사도는 상대적으로 낮다.

In [25]:
model.wv.similarity('히어로', '평론가')

0.6637813

'평론가'와 비슷한 단어들

In [26]:
model.wv.most_similar('평론가')

[('전문가', 0.9911670088768005),
 ('평론가들', 0.9909869432449341),
 ('높은거야', 0.9896015524864197),
 ('점이나', 0.9895164966583252),
 ('점대가', 0.9892258644104004),
 ('평론', 0.9883065223693848),
 ('점대는', 0.9882416129112244),
 ('점대라', 0.9879096746444702),
 ('높은거지', 0.9871199131011963),
 ('점이라', 0.9865758419036865)]

준비
학습된 FastText 모형을 불러온다.

In [27]:
from gensim.models.fasttext import FastText
ft = FastText.load('nsmc.fasttext')

데이터를 불러온다.

In [28]:
import pandas as pd
nsmc = pd.read_csv('ratings_train.txt', sep='\t')

전처리
리뷰가 있는 데이터만 선택한다.

In [29]:
df = nsmc[nsmc['document'].notnull()]

훈련용 데이터와 테스트용 데이터를 분할한다.

In [30]:
from sklearn.model_selection import train_test_split
doc_train, doc_test, y_train, y_test = train_test_split(df['document'], df['label'], test_size=0.2, random_state=42)

한글 단어만 추출하는 함수를 만든다.

In [31]:
import re
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

1000, 16 크기의 행렬을 만든다.

In [32]:
import numpy as np
x_train = np.zeros((1000, 16))

각 문서에서 한글 단어를 찾아 단어 임베딩을 구하고, 이를 문서마다 평균을 낸다.

In [33]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    vs = [ft.wv[word] for word in find_hangul(doc)]
    if vs:
        x_train[i,] = np.mean(vs, axis=0)

x_train은 각 문서의 단어 임베딩 평균이다.

In [33]:
x_train[0]

array([ 0.85029197,  1.39048409,  1.75145721,  0.75306934, -0.89818573,
       -0.17689162, -0.6556133 , -0.66011852,  1.11917162, -0.8815093 ,
       -0.95759803, -0.59692693,  0.25644588, -1.11521089, -0.30577666,
       -0.56151581])

모형 학습
각 문서의 단어 임베딩 평균을 이용하여 감성을 예측하는 모형을 만든다.

In [34]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(x_train, y_train.values[:1000], epochs=1)

32/32 [==============================] - 0s 656us/step - loss: 0.6874 - accuracy: 0.5300


In [1]:
import pandas as pd
df = pd.read_csv('imdb.zip')

In [2]:
import joblib
tk = joblib.load('tokenizer.pkl')

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
review_train, review_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

In [5]:
seqs = tk.texts_to_sequences(review_train)

In [9]:
seqs[0], review_train.iloc[0]

([9, 6, 33, 1258, 214], 'It is an insane game.')

순방향 순환신경망

In [6]:
import tensorflow as tf

패딩을 한다. 길이가 짧으면 앞쪽에 0을 채운다(padding='pre'). maxlen은 최대 길이를 지정할 수 있다. 지정하지 않으면 가장 긴 문자열의 길이로 지정된다. truncating='pre'는 maxlen보다 긴 문자열일 경우 앞쪽을 자른다. 뒤쪽을 자르게 하려면 'post'로 설정한다.

In [10]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=None, padding='pre', truncating='pre')

In [11]:
NUM_WORDS= tk.num_words + 1

Embedding에서 mask_zero=True로 설정하면 0으로 패딩된 부분의 예측은 손실에 반영하지 않는다.

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 8)           16008     
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 544       
_________________________________________________________________
dense (Dense)                (None, 1)                 9         
Total params: 16,561
Trainable params: 16,561
Non-trainable params: 0
_________________________________________________________________


모형을 학습시킨다.

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6930 - accuracy: 0.5200
Epoch 2/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6910 - accuracy: 0.6425
Epoch 3/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6859 - accuracy: 0.7675
Epoch 4/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6684 - accuracy: 0.8175
Epoch 5/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6014 - accuracy: 0.8325
Epoch 6/10
25/25 [==============================] - 0s 13ms/step - loss: 0.4919 - accuracy: 0.8363
Epoch 7/10
25/25 [==============================] - 0s 13ms/step - loss: 0.3968 - accuracy: 0.8900
Epoch 8/10
25/25 [==============================] - 0s 13ms/step - loss: 0.3273 - accuracy: 0.9225
Epoch 9/10
25/25 [==============================] - 0s 13ms/step - loss: 0.2771 - accuracy: 0.9388
Epoch 10/10
25/25 [==============================] - 0s 13ms/step - loss: 0.2338 - accuracy: 0.9613


역방향 순환신경망

패딩을 한다. 길이가 짧으면 뒤쪽에 0을 채운다(padding='post')

In [22]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, padding='post')

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6928 - accuracy: 0.5088
Epoch 2/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6892 - accuracy: 0.6650
Epoch 3/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6777 - accuracy: 0.7663
Epoch 4/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6331 - accuracy: 0.8025
Epoch 5/10
25/25 [==============================] - 0s 14ms/step - loss: 0.5461 - accuracy: 0.8612
Epoch 6/10
25/25 [==============================] - 0s 14ms/step - loss: 0.4421 - accuracy: 0.9025
Epoch 7/10
25/25 [==============================] - 0s 14ms/step - loss: 0.3816 - accuracy: 0.9112
Epoch 8/10
25/25 [==============================] - 0s 13ms/step - loss: 0.3153 - accuracy: 0.9362
Epoch 9/10
25/25 [==============================] - 0s 13ms/step - loss: 0.2833 - accuracy: 0.9450
Epoch 10/10
25/25 [==============================] - 0s 13ms/step - loss: 0.2560 - accuracy: 0.9463


양방향 순환신경망

LSTM을 Bidirectional로 감싸주면 자동으로 순방향과 역방향 레이어를 넣어준다.

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [27]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 8)           16008     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 16)                1088      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 17,113
Trainable params: 17,113
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 15ms/step - loss: 0.6921 - accuracy: 0.5362
Epoch 2/10
25/25 [==============================] - 0s 15ms/step - loss: 0.6849 - accuracy: 0.7013
Epoch 3/10
25/25 [==============================] - 0s 15ms/step - loss: 0.6648 - accuracy: 0.8037
Epoch 4/10
25/25 [==============================] - 0s 15ms/step - loss: 0.5980 - accuracy: 0.8388
Epoch 5/10
25/25 [==============================] - 0s 15ms/step - loss: 0.4443 - accuracy: 0.8763
Epoch 6/10
25/25 [==============================] - 0s 15ms/step - loss: 0.2928 - accuracy: 0.9150
Epoch 7/10
25/25 [==============================] - 0s 15ms/step - loss: 0.2127 - accuracy: 0.9513
Epoch 8/10
25/25 [==============================] - 0s 15ms/step - loss: 0.1689 - accuracy: 0.9613
Epoch 9/10
25/25 [==============================] - 0s 15ms/step - loss: 0.1410 - accuracy: 0.9663
Epoch 10/10
25/25 [==============================] - 0s 15ms/step - loss: 0.1157 - accuracy: 0.9787
